In [5]:
!pip install uv
!uv pip install -qU xpander-sdk llama-index llama-index-llms-openai python-dotenv

In [6]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

In [7]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [8]:
def square(a: int) -> int:
    """Multiply a by itself and returns the result integer"""
    return a*a

square_tool = FunctionTool.from_defaults(fn=square)

In [9]:
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools([multiply_tool, add_tool, square_tool], llm=llm, verbose=True)

In [10]:
response_gen = agent.stream_chat("What is 20+2*4? Calculate step by step")
response_gen.print_response_stream()

> Running step 7a3110dc-22db-487f-92f2-6cfea0bb5594. Step input: What is 20+2*4? Calculate step by step
Thought: The user wants to calculate the expression 20+2*4 step by step. According to the order of operations (BIDMAS/BODMAS), multiplication should be done before addition. So, I will first multiply 2 and 4.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step a1a0266d-098c-46cc-a56b-1f562a77cdb4. Step input: None
Thought: The multiplication of 2 and 4 is 8. Now, I will add this result to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step d9ad6c6d-5c2b-44d7-a2f3-4de6bd755700. Step input: None
 The result of the calculation 20+2*4 is 28.

In [11]:
response_gen_2 = agent.stream_chat("What is the square of 20? Calculate step by step")
response_gen_2.print_response_stream()

> Running step 9f386b88-c2dc-41fc-87e5-d36d97798ee6. Step input: What is the square of 20? Calculate step by step
Thought: The user wants to know the square of 20. I can use the 'square' tool to calculate this.
Action: square
Action Input: {'a': 20}
Observation: 400
> Running step a9396afc-c1b8-4d42-9529-e3f3a9620248. Step input: None
 The square of 20 is 400.

In [12]:
response_gen_3 = agent.stream_chat("What is the square of 14 + 12 squared? Calculate step by step")

> Running step 00c6949e-478b-4e69-83b5-fb9157794f9e. Step input: What is the square of 14 + 12 squared? Calculate step by step
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step edff563d-2df2-4189-aa35-f943b110b932. Step input: None
Thought: I have the square of 12, which is 144. Now I need to add 14 to this result.
Action: add
Action Input: {'a': 14, 'b': 144}
Observation: 158
> Running step 4132c27e-c325-44bb-9f23-b8c989cd6272. Step input: None
Thought: I have the result of 14 + 12 squared, which is 158. Now I need to find the square of this result.
Action: square
Action Input: {'a': 158}
Observation: 24964
> Running step 82c21af0-dfa5-4b46-b954-6388c704780a. Step input: None


In [13]:
response_gen_3.print_response_stream()

 The square of (14 + 12 squared) is 24964.

In [14]:
response_gen_4 = agent.stream_chat("What is the square of 14 + the square of 12? Calculate step by step")
response_gen_4.print_response_stream()

> Running step a3e18329-4742-4917-82b8-41578dd56fe0. Step input: What is the square of 14 + the square of 12? Calculate step by step
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: square
Action Input: {'a': 14}
Observation: 196
> Running step 3ef3ad31-fd96-4104-b311-53614880a7b7. Step input: None
Thought: I have the square of 14. Now I need to calculate the square of 12.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step 18121f6b-5779-44d9-9d83-b531a3902184. Step input: None
Thought: I have the square of 12. Now I need to add the square of 14 and the square of 12.
Action: add
Action Input: {'a': 196, 'b': 144}
Observation: 340
> Running step 21a21b1d-945b-4899-8b88-29a662af9556. Step input: None
 The square of 14 plus the square of 12 is 340.

In [15]:
from openai import OpenAI
from xpander_sdk import XpanderClient, LLMProvider, OpenAISupportedModels
# load the client
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")

# get the agent
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

xpander_agent.add_local_tools([multiply_tool, add_tool, square_tool])
print(xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI))

AttributeError: type object 'FunctionTool' has no attribute '__jsii_type__'

In [ ]:
# create execution
xpander_agent.invoke_agent("What is the square of 14 + 12 squared? Calculate step by step")

# init the memory
xpander_agent.memory.select_llm_provider(llm_provider=LLMProvider.OPEN_AI)
xpander_agent.memory.initialize_thread(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)

while not xpander_agent.is_finished():
    print("-"*100)
    print("Messages:",xpander_agent.memory.retrieve_messages())
    print("Tools:",xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI))
    print("-"*100)
    response = openai_client.chat.completions.create(
                model= OpenAISupportedModels.GPT_4_O,
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
                tool_choice="auto",
                temperature=0.0
        )
            
    # add messages directly from the LLM response
    xpander_agent.memory.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump(),llm_provider=LLMProvider.OPEN_AI)
    # run tools
    xpander_agent.run_tools(tool_calls=tool_calls)

# result (re fetch execution result)
# IMPORTANT: LAST TOOL IS xpfinish-agent-execution-finished WHICH IS AGENT-END with PARSING and may be slower due to inference times
execution_result = xpander_agent.execution.fetch(agent=xpander_agent,execution_id=xpander_agent.execution.id)
print("status", execution_result.status)
print("result", execution_result.result)